In [ ]:
from langchain_core.tools import tool, InjectedToolArg
import requests
from typing import Annotated

### Tool Creation

In [2]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """This function fetchs the currency conversion factor between a given base currency and a target currency"""
    url = f"https://v6.exchangerate-api.com/v6/522707655820473d385f4984/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Given a currency conversion rate, this function calculates the target currency value from a base currency value"""
    return base_currency_value * conversion_rate

In [3]:
get_conversion_factor.invoke({"base_currency": "INR", "target_currency": "USD"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1751846402,
 'time_last_update_utc': 'Mon, 07 Jul 2025 00:00:02 +0000',
 'time_next_update_unix': 1751932802,
 'time_next_update_utc': 'Tue, 08 Jul 2025 00:00:02 +0000',
 'base_code': 'INR',
 'target_code': 'USD',
 'conversion_rate': 0.01169}

### Tool Binding

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

d:\Gen AI\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage("What is conversion rate from INR to USD, and based on that convert 98745 inr to USD")]

In [14]:
ai_message = llm_with_tools.invoke(messages)

ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '44be6471-8b2d-42bf-afc7-396cbeea542d',
  'type': 'tool_call'}]

In [15]:
ai_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ccaa162c-37f1-4f90-98d2-5cf5e35606e3-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '44be6471-8b2d-42bf-afc7-396cbeea542d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 26, 'total_tokens': 908, 'input_token_details': {'cache_read': 0}})

In [16]:
messages.append(ai_message)

messages

[HumanMessage(content='What is conversion rate from INR to USD, and based on that convert 98745 inr to USD', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ccaa162c-37f1-4f90-98d2-5cf5e35606e3-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '44be6471-8b2d-42bf-afc7-396cbeea542d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 26, 'total_tokens': 908, 'input_token_details': {'cache_read': 0}})]

In [17]:
tool_call = get_conversion_factor.invoke(ai_message.tool_calls[0])

In [18]:
import json
json.loads(tool_call.content)["conversion_rate"]

0.01169

In [19]:
messages.append(tool_call)
messages

[HumanMessage(content='What is conversion rate from INR to USD, and based on that convert 98745 inr to USD', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ccaa162c-37f1-4f90-98d2-5cf5e35606e3-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': '44be6471-8b2d-42bf-afc7-396cbeea542d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 26, 'total_tokens': 908, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1751846402, "time_last

In [20]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'convert', 'arguments': '{"base_currency_value": 98745.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--3fcf2293-0b40-4848-a34d-815af38bce81-0', tool_calls=[{'name': 'convert', 'args': {'base_currency_value': 98745.0}, 'id': '421a88a3-e16e-4f41-b54b-5da49873f0d9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 378, 'output_tokens': 21, 'total_tokens': 580, 'input_token_details': {'cache_read': 0}})

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

In [22]:
from langchain.agents import initialize_agent, AgentType

agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, # using REACT pattern
    verbose=True # shows internal thinking
)

In [23]:
query = "Hi how are you"
response = agent_executor.invoke({"input": query})

response



> Entering new AgentExecutor chain...
Thought: The user is greeting me. I should respond in a friendly manner. There is no need to use any tools for this.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm doing great, thank you for asking! How can I help you today?"
}
```

> Finished chain.


{'input': 'Hi how are you',
 'output': "I'm doing great, thank you for asking! How can I help you today?"}

In [24]:
response = agent_executor.invoke({"input": "what is the conversion factor from USD to INR and based on that convert 96 INR to USD"})



> Entering new AgentExecutor chain...
Thought: The user is asking for two things:
1. The conversion factor from USD to INR.
2. Convert 96 INR to USD based on that factor.

I need to use the `get_conversion_factor` tool first to get the conversion factor from USD to INR.
Then, I'll need to use the `convert` tool. However, the `convert` tool takes `base_currency_value` and `conversion_rate`. The `get_conversion_factor` returns the rate from `base_currency` to `target_currency`. If I get the rate from USD to INR, say it's X, then 1 USD = X INR. To convert INR to USD, I'll need to divide by X or multiply by 1/X.

Let's start by getting the conversion factor from USD to INR.
Action:
```json
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "USD",
    "target_currency": "INR"
  }
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1

In [25]:
response

{'input': 'what is the conversion factor from USD to INR and based on that convert 96 INR to USD',
 'output': 'The conversion factor from USD to INR is 85.543. Based on this, 96 INR is approximately 1.12 USD.'}

In [26]:
response = agent_executor.invoke({"input": "what is the conversion factor from INR to USD and based on that convert 96 INR to USD"})



> Entering new AgentExecutor chain...
Thought: The user is asking for two things:
1. The conversion factor from INR to USD.
2. The conversion of 96 INR to USD based on that factor.

I need to use the `get_conversion_factor` tool first to get the rate, and then the `convert` tool to perform the conversion.
Action:
```json
{
  "action": "get_conversion_factor",
  "action_input": {
    "base_currency": "INR",
    "target_currency": "USD"
  }
}
```
Observation: {'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1751846402, 'time_last_update_utc': 'Mon, 07 Jul 2025 00:00:02 +0000', 'time_next_update_unix': 1751932802, 'time_next_update_utc': 'Tue, 08 Jul 2025 00:00:02 +0000', 'base_code': 'INR', 'target_code': 'USD', 'conversion_rate': 0.01169}
Thought:Thought: I have successfully retrieved the conversion factor from INR to USD, which is 0.01169. Now I need to use this factor to

In [27]:
response

{'input': 'what is the conversion factor from INR to USD and based on that convert 96 INR to USD',
 'output': 'The conversion factor from INR to USD is 0.01169.\n96 INR is equal to 1.12224 USD.'}